# Estimating model energy

[![Open In
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nengo/keras-spiking/blob/master/docs/examples/model-energy.ipynb)

One of the main motivations for using spiking methods is the potential for significant
energy savings over standard techniques. Thus it is useful to be able to estimate how
much energy would be used by a model on different devices, so that we can get an
idea of how different model/device parameters affect the energy usage before pursuing a
full deployment.

In [ ]:
!pip install keras_spiking

In [13]:
import warnings

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import TensorBoard

import keras_spiking

warnings.simplefilter("ignore")
tf.get_logger().addFilter(lambda rec: "Tracing is expensive" not in rec.msg)

## Using ModelEnergy

The ``keras_spiking.ModelEnergy`` class provides the entry point for energy estimation.
It takes a Keras model as input, and computes relevant statistics for that model.

LCRM-VGG9 for OCR on emnist


In [14]:
import tensorflow as tf

inp = x = tf.keras.Input((224, 224, 1))
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(11, 11), strides=(4, 4))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=5, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1, kernel_size=3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=2, kernel_size=3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1, kernel_size=3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(62, activation='softmax')(x)  # Change to the number of classes you want

model = tf.keras.Model(inp, x)
model.summary()



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 54, 54, 64)        7808      
                                                                 
 batch_normalization (Batch  (None, 54, 54, 64)        256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 64)        0         
 D)                                                              
                                                                 
 depthwise_conv2d (Depthwis  (None, 26, 26, 128)       3328

In [15]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model)
energy.summary(print_warnings=False)

Layer (type)                           |Output shape       |Param #|Conn #   |Neuron #|J/inf (cpu)
---------------------------------------|-------------------|-------|---------|--------|-----------
input_2 (InputLayer)                   |[(None, 224, 224, 1|      0|        0|       0|          0
conv2d_1 (Conv2D)                      | (None, 54, 54, 64)|   7808| 22581504|       0|       0.19
batch_normalization (BatchNormalization| (None, 54, 54, 64)|    256|        0|       0|          0
activation (Activation)                | (None, 54, 54, 64)|      0|        0|  186624|     0.0016
max_pooling2d (MaxPooling2D)           | (None, 26, 26, 64)|      0|        0|       0|          0
depthwise_conv2d (DepthwiseConv2D)     |(None, 26, 26, 128)|   3328|        0|       0|          0
conv2d_2 (Conv2D)                      |(None, 26, 26, 128)| 409728|276889600|       0|        2.4
batch_normalization_1 (BatchNormalizati|(None, 26, 26, 128)|    512|        0|       0|          0
activation

In [16]:
energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)      |J/inf (cpu)|J/inf (gpu)|Synop J/inf (|Synop J/inf (|Neuron J/inf |Neuron J/inf 
------------------|-----------|-----------|-------------|-------------|-------------|-------------
input_2 (InputLaye|          0|          0|            0|            0|            0|            0
conv2d_1 (Conv2D) |       0.19|     0.0068|         0.19|       0.0068|            0|            0
batch_normalizatio|          0|          0|            0|            0|            0|            0
activation (Activa|     0.0016|    5.6e-05|            0|            0|       0.0016|      5.6e-05
max_pooling2d (Max|          0|          0|            0|            0|            0|            0
depthwise_conv2d (|          0|          0|            0|            0|            0|            0
conv2d_2 (Conv2D) |        2.4|      0.083|          2.4|        0.083|            0|            0
batch_normalizatio|          0|          0|            0|            0|            0|            0
activation

VGG 9 for OCR on EMNIST

In [22]:
import tensorflow as tf

inp = x = tf.keras.Input((224, 224, 1))
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(11, 11), strides=(4, 4), input_shape=(224, 224, 1))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(100, activation='softmax')(x)

model2 = tf.keras.Model(inp, x)
model2.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 54, 54, 64)        7808      
                                                                 
 batch_normalization_6 (Bat  (None, 54, 54, 64)        256       
 chNormalization)                                                
                                                                 
 activation_6 (Activation)   (None, 54, 54, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 26, 26, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 26, 26, 128)       2049

In [23]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model2)
energy.summary(print_warnings=False)

Layer (type)                           |Output shape       |Param #|Conn #   |Neuron #|J/inf (cpu)
---------------------------------------|-------------------|-------|---------|--------|-----------
input_3 (InputLayer)                   |[(None, 224, 224, 1|      0|        0|       0|          0
conv2d_3 (Conv2D)                      | (None, 54, 54, 64)|   7808| 22581504|       0|       0.19
batch_normalization_6 (BatchNormalizati| (None, 54, 54, 64)|    256|        0|       0|          0
activation_6 (Activation)              | (None, 54, 54, 64)|      0|        0|  186624|     0.0016
max_pooling2d_4 (MaxPooling2D)         | (None, 26, 26, 64)|      0|        0|       0|          0
conv2d_4 (Conv2D)                      |(None, 26, 26, 128)| 204928|138444800|       0|        1.2
batch_normalization_7 (BatchNormalizati|(None, 26, 26, 128)|    512|        0|       0|          0
activation_7 (Activation)              |(None, 26, 26, 128)|      0|        0|   86528|    0.00074
max_poolin

In [24]:
energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)      |J/inf (cpu)|J/inf (gpu)|Synop J/inf (|Synop J/inf (|Neuron J/inf |Neuron J/inf 
------------------|-----------|-----------|-------------|-------------|-------------|-------------
input_3 (InputLaye|          0|          0|            0|            0|            0|            0
conv2d_3 (Conv2D) |       0.19|     0.0068|         0.19|       0.0068|            0|            0
batch_normalizatio|          0|          0|            0|            0|            0|            0
activation_6 (Acti|     0.0016|    5.6e-05|            0|            0|       0.0016|      5.6e-05
max_pooling2d_4 (M|          0|          0|            0|            0|            0|            0
conv2d_4 (Conv2D) |        1.2|      0.042|          1.2|        0.042|            0|            0
batch_normalizatio|          0|          0|            0|            0|            0|            0
activation_7 (Acti|    0.00074|    2.6e-05|            0|            0|      0.00074|      2.6e-05
max_poolin

# **Result Summary of LCRM (optimized) VGG9:**



Total params: 3654462 (13.94 MB)

Trainable params: 3651006 (13.93 MB)

Non-trainable params: 3456 (13.50 KB)




Total energy per inference [Joules/inf] (cpu): 2.61e+00

Total energy per inference [Joules/inf] (gpu): 9.09e-02

# **Result Summary of VGG9 (parent model):**
Total params: 7895140 (30.12 MB)

Trainable params: 7891684 (30.10 MB)

Non-trainable params: 3456 (13.50 KB)


Total energy per inference [Joules/inf] (cpu): 4.48e+00

Total energy per inference [Joules/inf] (gpu): 1.56e-01


**Overall there is a 42 % reduction in total energy perinference (GPU) and 54 % reduction in storage requirement**





AlexNet Optimized model for image classification

In [30]:
import tensorflow as tf

input = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.layers.Conv2D(64, kernel_size=11, strides=(4, 4))(input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)


x = layers.DepthwiseConv2D(depth_multiplier=4,kernel_size=5,strides=(1,1),padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

# Middle loop
x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1, kernel_size=3, strides=(1, 1), padding='same')(x1)
x = tf.keras.layers.BatchNormalization()(x)
x2 = tf.keras.layers.Activation('relu')(x)

x = tf.keras.layers.Conv2D(128, kernel_size=1, strides=(1, 1), padding='same')(x1)
x = tf.keras.layers.BatchNormalization()(x)
x3 = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Concatenate()([x2, x3])

x = tf.keras.layers.DepthwiseConv2D(depth_multiplier=1, kernel_size=3, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

# Exit flow
x = tf.keras.layers.Conv2D(256, kernel_size=1, strides=(1, 1), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(10, activation='softmax')(x)

model3 = tf.keras.Model(input, output)
model3.summary()


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_31 (Conv2D)          (None, 54, 54, 64)           23296     ['input_9[0][0]']             
                                                                                                  
 batch_normalization_39 (Ba  (None, 54, 54, 64)           256       ['conv2d_31[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_24 (Activation)  (None, 54, 54, 64)           0         ['batch_normalization_3

In [31]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model3)
energy.summary(print_warnings=False)

energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)                           |Output shape        |Param #|Conn #  |Neuron #|J/inf (cpu)
---------------------------------------|--------------------|-------|--------|--------|-----------
input_9 (InputLayer)                   |[(None, 224, 224, 3)|      0|       0|       0|          0
conv2d_31 (Conv2D)                     |  (None, 54, 54, 64)|  23296|67744512|       0|       0.58
batch_normalization_39 (BatchNormalizat|  (None, 54, 54, 64)|    256|       0|       0|          0
activation_24 (Activation)             |  (None, 54, 54, 64)|      0|       0|  186624|     0.0016
max_pooling2d_23 (MaxPooling2D)        |  (None, 26, 26, 64)|      0|       0|       0|          0
depthwise_conv2d_10 (DepthwiseConv2D)  | (None, 26, 26, 256)|   6656|       0|       0|          0
batch_normalization_40 (BatchNormalizat| (None, 26, 26, 256)|   1024|       0|       0|          0
activation_25 (Activation)             | (None, 26, 26, 256)|      0|       0|  173056|     0.0015
max_poolin

AlexNet (Parent model)

In [32]:

inp = x = tf.keras.Input((227, 227, 3))
x = tf.keras.layers.Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same")(x)
x = tf.keras.layers.BatchNormalization()(x)

x = tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(10, activation='softmax')(x)

model4 = tf.keras.Model(inp, x)
model4.summary()


Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 227, 227, 3)]     0         
                                                                 
 conv2d_34 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 batch_normalization_45 (Ba  (None, 55, 55, 96)        384       
 tchNormalization)                                               
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 27, 27, 96)        0         
 ng2D)                                                           
                                                                 
 conv2d_35 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 batch_normalization_46 (Ba  (None, 27, 27, 256)       102

In [33]:
# estimate model energy
energy = keras_spiking.ModelEnergy(model4)
energy.summary(print_warnings=False)

energy.summary(
    columns=(
        "name",
        "energy cpu",
        "energy gpu",
        "synop_energy cpu",
        "synop_energy gpu",
        "neuron_energy cpu",
        "neuron_energy gpu",
    ),
    print_warnings=False,
)

Layer (type)                           |Output shape       |Param #|Conn #   |Neuron #|J/inf (cpu)
---------------------------------------|-------------------|-------|---------|--------|-----------
input_10 (InputLayer)                  |[(None, 227, 227, 3|      0|        0|       0|          0
conv2d_34 (Conv2D)                     | (None, 55, 55, 96)|  34944|105415200|  290400|       0.91
batch_normalization_45 (BatchNormalizat| (None, 55, 55, 96)|    384|        0|       0|          0
max_pooling2d_26 (MaxPooling2D)        | (None, 27, 27, 96)|      0|        0|       0|          0
conv2d_35 (Conv2D)                     |(None, 27, 27, 256)| 614656|447897600|  186624|        3.9
batch_normalization_46 (BatchNormalizat|(None, 27, 27, 256)|   1024|        0|       0|          0
max_pooling2d_27 (MaxPooling2D)        |(None, 13, 13, 256)|      0|        0|       0|          0
conv2d_36 (Conv2D)                     |(None, 13, 13, 384)| 885120|149520384|   64896|        1.3
batch_norm

# **Result Summary of LCRM (optimized) AlexNEt:**
Total params: 7787658 (29.71 MB)

Trainable params: 7784970 (29.70 MB)

Non-trainable params: 2688 (10.50 KB)

Total energy per inference [Joules/inf] (cpu): 8.15e-01

Total energy per inference [Joules/inf] (gpu): 2.84e-02

# **Result Summary of parent ALexNet model:**
Total params: 14250762 (54.36 MB)

Trainable params: 14248010 (54.35 MB)

Non-trainable params: 2752 (10.75 KB)

Total energy per inference [Joules/inf] (cpu): 9.35e+00

Total energy per inference [Joules/inf] (gpu): 3.26e-01



**Overall there is a 91.2 % reduction in total energy perinference (GPU) and 45.3 % reduction in storage requirement**